# DKASC数据预处理第二版程序

程序从原始下载数据中筛选用于训练集和测试集，输出

In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
# import tensorflow as tf
# import keras
#import torch
# import catboost as cb
from pandas import read_csv
from math import sqrt
# from numpy import concatenate
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.layers import LSTM
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_squared_error
# from keras.layers.core import Dense, Activation, Dropout
%matplotlib inline

In [2]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# config = tf.ConfigProto(allow_soft_placement = True)
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.5)
# config.gpu_options.allow_growth = True
# sess0 = tf.InteractiveSession(config = config)

In [3]:
print(os.getcwd()) # 打印当前工作目录

E:\Research Project\DKASC, Alice Springs


In [4]:
os.listdir(os.getcwd())

['.idea',
 '.ipynb_checkpoints',
 '101-Site_DKA-WeatherStation',
 '96-Site_DKA-MasterMeter1',
 'Alice_SpringsLSTM.ipynb',
 'Alice_SpringsWPD-LSTM.ipynb',
 'Alice_Springs_2016-2020',
 'Dataset',
 'logs',
 'picture',
 'Predict',
 'Program backups',
 'temp',
 'venv',
 'WPD.ipynb',
 '数据集预处理-第二版0702.ipynb',
 '模型测试.ipynb',
 '模型训练0702.ipynb',
 '画图.ipynb',
 '笔记.ipynb',
 '训练模型多模型.ipynb']

In [5]:
os.chdir("./Alice_Springs_2016-2020/")

In [6]:
os.listdir()

['3D.jpg',
 'Alice_Springs_2016.csv',
 'Alice_Springs_2017.csv',
 'Alice_Springs_2018.csv',
 'Alice_Springs_2019.csv',
 'Alice_Springs_2020.csv']

In [7]:
dataset1 = pd.read_csv('Alice_Springs_2016.csv')
dataset2 = pd.read_csv('Alice_Springs_2017.csv')
dataset3 = pd.read_csv('Alice_Springs_2018.csv')
dataset4 = pd.read_csv('Alice_Springs_2019.csv')
# test = pd.read_csv('Alice_Springs_2020.csv')


In [8]:
#查看输入数据维度
print(dataset1.shape)
print(dataset2.shape)
print(dataset3.shape)
print(dataset4.shape)

(105462, 66)
(105155, 52)
(105229, 53)
(105321, 53)


In [9]:
dataset = pd.concat([dataset1,dataset2,dataset3,dataset4],axis=0)
# dataset = pd.concat([dataset3,dataset4],axis=0)    #连接两个训练集 并查看数据维度

# dataset.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
print(dataset.shape)

(421167, 70)


In [11]:
dataset.head()

10 SunPower - Active Power (kW)  11 BP Solar - Active Power (kW)  \
0                              0.0                              0.0   
1                              0.0                              0.0   
2                              0.0                              0.0   
3                              0.0                              0.0   
4                              0.0                              0.0   

   12 BP Solar - Active Power (kW)  13 Trina - Active Power (kW)  \
0                              0.0                           0.0   
1                              0.0                           0.0   
2                              0.0                           0.0   
3                              0.0                           0.0   
4                              0.0                           0.0   

   14 Kyocera - Active Power (kW)  14 Kyocera - Performance Ratio (%)  \
0                             0.0                                 0.0   
1                             0.0                                 0.0   
2                             0.0                                 0.0   
3                             0.0                                 0.0   
4                             0.0                                 0.0   

   16A BP Solar - Active Power (kW)  16B BP Solar - Active Power (kW)  \
0                               0.0                               0.0   
1                               0.0                               0.0   
2                               0.0                               0.0   
3                               0.0                               0.0   
4                               0.0                               0.0   

   16C BP Solar - Active Power (kW)  16D BP Solar - Active Power (kW)  ...  \
0                               0.0                               0.0  ...   
1                               0.0                               0.0  ...   
2                               0.0                               0.0  ...   
3                               0.0                               0.0  ...   
4                               0.0                               0.0  ...   

   DKA.WeatherStation - Diffuse Horizontal Radiation (W/m²)  \
0                                           1.103375          
1                                           1.152340          
2                                           1.566642          
3                                           1.707677          
4                                           1.746989          

   DKA.WeatherStation - Global Horizontal Radiation (W/m²)  \
0                                           2.672255         
1                                           2.703700         
2                                           3.013785         
3                                           3.421924         
4                                           3.292158         

   DKA.WeatherStation - Radiation Diffuse Tilted (W/m²)  \
0                                           1.853108      
1                                           1.796472      
2                                           2.343814      
3                                           2.654432      
4                                           2.495872      

   DKA.WeatherStation - Radiation Global Tilted (W/m²)  \
0                                           1.868883     
1                                           1.803461     
2                                           2.339108     
3                                           2.719022     
4                                           2.489688     

   DKA.WeatherStation - Weather Daily Rainfall (mm)  \
0                                               0.0   
1                                               0.0   
2                                               0.0   
3                                               0.0   
4                                               0.0   

   DKA.WeatherStation - Weather R

# 数据预处理

设定索引为日期，方便按日期筛选

In [12]:
# 设定索引为日期，方便按日期筛选
dataset.set_index('Timestamp', inplace=True,drop=False)
# test.set_index('Timestamp', inplace=True,drop=False)

In [13]:
dataset.head()

10 SunPower - Active Power (kW)  \
Timestamp                                        
2016/1/1 0:00                              0.0   
2016/1/1 0:05                              0.0   
2016/1/1 0:10                              0.0   
2016/1/1 0:15                              0.0   
2016/1/1 0:20                              0.0   

               11 BP Solar - Active Power (kW)  \
Timestamp                                        
2016/1/1 0:00                              0.0   
2016/1/1 0:05                              0.0   
2016/1/1 0:10                              0.0   
2016/1/1 0:15                              0.0   
2016/1/1 0:20                              0.0   

               12 BP Solar - Active Power (kW)  13 Trina - Active Power (kW)  \
Timestamp                                                                      
2016/1/1 0:00                              0.0                           0.0   
2016/1/1 0:05                              0.0                           0.0   
2016/1/1 0:10                              0.0                           0.0   
2016/1/1 0:15                              0.0                           0.0   
2016/1/1 0:20                              0.0                           0.0   

               14 Kyocera - Active Power (kW)  \
Timestamp                                       
2016/1/1 0:00                             0.0   
2016/1/1 0:05                             0.0   
2016/1/1 0:10                             0.0   
2016/1/1 0:15                             0.0   
2016/1/1 0:20                             0.0   

               14 Kyocera - Performance Ratio (%)  \
Timestamp                                           
2016/1/1 0:00                                 0.0   
2016/1/1 0:05                                 0.0   
2016/1/1 0:10                                 0.0   
2016/1/1 0:15                                 0.0   
2016/1/1 0:20                                 0.0   

               16A BP Solar - Active Power (kW)  \
Timestamp                                         
2016/1/1 0:00                               0.0   
2016/1/1 0:05                               0.0   
2016/1/1 0:10                               0.0   
2016/1/1 0:15                               0.0   
2016/1/1 0:20                               0.0   

               16B BP Solar - Active Power (kW)  \
Timestamp                                         
2016/1/1 0:00                               0.0   
2016/1/1 0:05                               0.0   
2016/1/1 0:10                               0.0   
2016/1/1 0:15                               0.0   
2016/1/1 0:20                               0.0   

               16C BP Solar - Active Power (kW)  \
Timestamp                                         
2016/1/1 0:00                               0.0   
2016/1/1 0:05                               0.0   
2016/1/1 0:10                               0.0   
2016/1/1 0:15                               0.0   
2016/1/1 0:20                               0.0   

               16D BP Solar - Active Power (kW)  ...  \
Timestamp                                        ...   
2016/1/1 0:00                               0.0  ...   
2016/1/1 0:05                               0.0  ...   
2016/1/1 0:10                               0.0  ...   
2016/1/1 0:15                               0.0  ...   
2016/1/1 0:20                               0.0  ...   

               DKA.WeatherStation - Diffuse Horizontal Radiation (W/m²)  \
Timestamp                                                                 
2016/1/1 0:00                                           1.103375          
2016/1/1 0:05                                           1.152340          
2016/1/1 0:10                                           1.566642          
2016/1/1 0:15                                           1.707677          
2016/1/1 0:20                                           1.746989          

               DKA.WeatherStation - Global 

训练集除空

In [14]:
print('训练集初始维度：',dataset.shape)
dataset.dropna(axis="columns", how='all', inplace=True)  #删除掉训练集全是空值的列
dataset.dropna(axis="index", how='all', inplace=True)   #删除掉训练集全是空值的行
print('训练集去除全空维度：',dataset.shape)
dataset

训练集初始维度： (421167, 70)
训练集去除全空维度： (421167, 69)


10 SunPower - Active Power (kW)  \
Timestamp                                           
2016/1/1 0:00                                 0.0   
2016/1/1 0:05                                 0.0   
2016/1/1 0:10                                 0.0   
2016/1/1 0:15                                 0.0   
2016/1/1 0:20                                 0.0   
2016/1/1 0:25                                 0.0   
2016/1/1 0:30                                 0.0   
2016/1/1 0:35                                 0.0   
2016/1/1 0:40                                 0.0   
2016/1/1 0:45                                 0.0   
2016/1/1 0:50                                 0.0   
2016/1/1 0:55                                 0.0   
2016/1/1 1:00                                 0.0   
2016/1/1 1:05                                 0.0   
2016/1/1 1:10                                 0.0   
2016/1/1 1:15                                 0.0   
2016/1/1 1:20                                 0.0   
2016/1/1 1:25                                 0.0   
2016/1/1 1:30                                 0.0   
2016/1/1 1:35                                 0.0   
2016/1/1 1:40                                 0.0   
2016/1/1 1:45                                 0.0   
2016/1/1 1:50                                 0.0   
2016/1/1 1:55                                 0.0   
2016/1/1 2:00                                 0.0   
2016/1/1 2:05                                 0.0   
2016/1/1 2:10                                 0.0   
2016/1/1 2:15                                 0.0   
2016/1/1 2:20                                 0.0   
2016/1/1 2:25                                 0.0   
...                                           ...   
2019/12/31 21:30                              0.0   
2019/12/31 21:35                              0.0   
2019/12/31 21:40                              0.0   
2019/12/31 21:45                              0.0   
2019/12/31 21:50                              0.0   
2019/12/31 21:55                              0.0   
2019/12/31 22:00                              0.0   
2019/12/31 22:05                              0.0   
2019/12/31 22:10                              0.0   
2019/12/31 22:15                              0.0   
2019/12/31 22:20                              0.0   
2019/12/31 22:25                              0.0   
2019/12/31 22:30                              0.0   
2019/12/31 22:35                              0.0   
2019/12/31 22:40                              0.0   
2019/12/31 22:45                              0.0   
2019/12/31 22:50                              0.0   
2019/12/31 22:55                              0.0   
2019/12/31 23:00                              0.0   
2019/12/31 23:05                              0.0   
2019/12/31 23:10                              0.0   
2019/12/31 23:15                              0.0   
2019/12/31 23:20                              0.0   
2019/12/31 23:25                              0.0   
2019/12/31 23:30                              0.0   
2019/12/31 23:35                              0.0   
2019/12/31 23:40                              0.0   
2019/12/31 23:45                              0.0   
2019/12/31 23:50                              0.0   
2019/12/31 23:55                              0.0   

                  11 BP Solar - Active Power (kW)  \
Timestamp                                           
2016/1/1 0:00                                 0.0   
2016/1/1 0:05                                 0.0   
2016/1/1 0:10                                 0.0   
2016/1/1 0:15                                 0.0   
2016/1/1 0:20                                 0.0   
2016/1/1 0:25                                 0.0   
2016/1/1 0:30                                 0.0   
2016/1/1 0:35                                 0.0   
2016/1/1 0:40                                 0.0   
2016/1/1 0:45                                 0.0   
2016/1/1 0:50                             

In [15]:
# 查看训练数据的形状，列名索引、数据类型
print('数据集维度:\n',dataset.shape)
print('\n数据集列名：\n',dataset.columns)
print('\n数据集索引列：\n',dataset.index)
print('\n数据集每列的数据类型：',dataset.dtypes)

数据集维度:
 (421167, 69)

数据集列名：
 Index(['10 SunPower - Active Power (kW)', '11 BP Solar - Active Power (kW)',
       '12 BP Solar - Active Power (kW)', '13 Trina - Active Power (kW)',
       '14 Kyocera - Active Power (kW)', '14 Kyocera - Performance Ratio (%)',
       '16A BP Solar - Active Power (kW)', '16B BP Solar - Active Power (kW)',
       '16C BP Solar - Active Power (kW)', '16D BP Solar - Active Power (kW)',
       '17 Sanyo - Active Power (kW)', '18 SunPower - Active Power (kW)',
       '19 Sungrid - Active Power (kW)', '1A Trina - Active Power (kW)',
       '1B Trina - Active Power (kW)', '2 eco-Kinetics - Active Power (kW)',
       '20 Sungrid - Active Power (kW)',
       '21 Evergreen Solar - Active Power (kW)',
       '22 SolFocus - Active Power (kW)', '23 Calyxo - Active Power (kW)',
       '24 Hanwha Q-Cells - Active Power (kW)',
       '24 Q CELLS - Active Power (kW)', '25 Hanwha Solar - Active Power (kW)',
       '26 Hanwha Q-Cells - Active Power (kW)',
       '26 Q CELL

# 选取九个站点

In [16]:
dataset = dataset[['Timestamp',
       'DKA.WeatherStation - Weather Temperature Celsius (°C)',
       'DKA.WeatherStation - Weather Relative Humidity (%)',
       'DKA.WeatherStation - Wind Speed (m/s)',
       'DKA.WeatherStation - Wind Direction (Degrees)',
       'DKA.WeatherStation - Weather Daily Rainfall (mm)',
       'DKA.WeatherStation - Global Horizontal Radiation (W/m²)',
       'DKA.WeatherStation - Diffuse Horizontal Radiation (W/m²)',
       'DKA.WeatherStation - Radiation Global Tilted (W/m²)',
       'DKA.WeatherStation - Radiation Diffuse Tilted (W/m²)',
       '1A Trina - Active Power (kW)',
       '1B Trina - Active Power (kW)', '2 eco-Kinetics - Active Power (kW)',
       '3 BP Solar - Active Power (kW)', '4 Kyocera - Active Power (kW)',
       '5 Kyocera - Active Power (kW)', '6 Kyocera - Active Power (kW)',
       '7 First Solar - Active Power (kW)', '8 Kaneka - Active Power (kW)',
       'DKA.MasterMeter1 - Active Power (kW)',
       'DKA.MasterMeter2 - Active Power (kW)']]
print(dataset.head(5))

                   Timestamp  \
Timestamp                      
2016/1/1 0:00  2016/1/1 0:00   
2016/1/1 0:05  2016/1/1 0:05   
2016/1/1 0:10  2016/1/1 0:10   
2016/1/1 0:15  2016/1/1 0:15   
2016/1/1 0:20  2016/1/1 0:20   

               DKA.WeatherStation - Weather Temperature Celsius (°C)  \
Timestamp                                                              
2016/1/1 0:00                                          17.176111       
2016/1/1 0:05                                          17.133446       
2016/1/1 0:10                                          17.084999       
2016/1/1 0:15                                          16.955778       
2016/1/1 0:20                                          16.854271       

               DKA.WeatherStation - Weather Relative Humidity (%)  \
Timestamp                                                           
2016/1/1 0:00                                          87.095665    
2016/1/1 0:05                                          87.67166

In [17]:
#电站和气象数据重命名
dataset.rename(columns={
       'DKA.WeatherStation - Weather Temperature Celsius (°C)':'Temperature(°C)',
       'DKA.WeatherStation - Weather Relative Humidity (%)':'Relative Humidity(%)',
       'DKA.WeatherStation - Wind Speed (m/s)':'Wind Speed(m/s)',
       'DKA.WeatherStation - Wind Direction (Degrees)':'Wind Direction(Degrees)',
       'DKA.WeatherStation - Weather Daily Rainfall (mm)':'Daily Rainfall(mm)',
       'DKA.WeatherStation - Global Horizontal Radiation (W/m²)':'Global Horizontal Radiation(W/m²)',
       'DKA.WeatherStation - Diffuse Horizontal Radiation (W/m²)':'Diffuse Horizontal Radiation(W/m²)',
       'DKA.WeatherStation - Radiation Global Tilted (W/m²)':'Radiation Global Tilted(W/m²)',
       'DKA.WeatherStation - Radiation Diffuse Tilted (W/m²)':'Radiation Diffuse Tilted(W/m²)',
       '1A Trina - Active Power (kW)':'1A Trina_power',
       '1B Trina - Active Power (kW)':'1B Trina_power',
       '2 eco-Kinetics - Active Power (kW)':'2 eco_power',
       '3 BP Solar - Active Power (kW)':'3 BP_power',
       '4 Kyocera - Active Power (kW)':'4 Kyocera_power',
       '5 Kyocera - Active Power (kW)':'5 Kyocera_power',
       '6 Kyocera - Active Power (kW)':'6 Kyocera_power',
       '7 First Solar - Active Power (kW)':'7 First_power',
       '8 Kaneka - Active Power (kW)':'8 Kaneka_power',
       'DKA.MasterMeter1 - Active Power (kW)':'DKA.Meter1_power',
       'DKA.MasterMeter2 - Active Power (kW)':'DKA.Meter2_power'},inplace=True)

dataset.columns

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


Index(['Timestamp', 'Temperature(°C)', 'Relative Humidity(%)',
       'Wind Speed(m/s)', 'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Radiation Global Tilted(W/m²)',
       'Radiation Diffuse Tilted(W/m²)', '1A Trina_power', '1B Trina_power',
       '2 eco_power', '3 BP_power', '4 Kyocera_power', '5 Kyocera_power',
       '6 Kyocera_power', '7 First_power', '8 Kaneka_power',
       'DKA.Meter1_power', 'DKA.Meter2_power'],
      dtype='object')

In [18]:
dataset.rename(columns={ '1A Trina_power':'Station_1',
                        '1B Trina_power':'Station_2',
                        '2 eco_power':'Station_3',
                        '3 BP_power':'Station_4', 
                        '4 Kyocera_power':'Station_5', 
                        '5 Kyocera_power':'Station_6',
                        '6 Kyocera_power':'Station_7', 
                        '7 First_power':'Station_8', 
                        '8 Kaneka_power':'Station_9'
       },inplace=True)
dataset.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


Timestamp  Temperature(°C)  Relative Humidity(%)  \
Timestamp                                                             
2016/1/1 0:00  2016/1/1 0:00        17.176111             87.095665   
2016/1/1 0:05  2016/1/1 0:05        17.133446             87.671669   
2016/1/1 0:10  2016/1/1 0:10        17.084999             88.563240   
2016/1/1 0:15  2016/1/1 0:15        16.955778             90.001663   
2016/1/1 0:20  2016/1/1 0:20        16.854271             91.222733   

               Wind Speed(m/s)  Wind Direction(Degrees)  Daily Rainfall(mm)  \
Timestamp                                                                     
2016/1/1 0:00         2.421402               186.941284                 0.0   
2016/1/1 0:05         2.688986               191.876770                 0.0   
2016/1/1 0:10         2.672599               188.268082                 0.0   
2016/1/1 0:15         3.787027               197.727295                 0.0   
2016/1/1 0:20         3.345906               193.704559                 0.0   

               Global Horizontal Radiation(W/m²)  \
Timestamp                                          
2016/1/1 0:00                           2.672255   
2016/1/1 0:05                           2.703700   
2016/1/1 0:10                           3.013785   
2016/1/1 0:15                           3.421924   
2016/1/1 0:20                           3.292158   

               Diffuse Horizontal Radiation(W/m²)  \
Timestamp                                           
2016/1/1 0:00                            1.103375   
2016/1/1 0:05                            1.152340   
2016/1/1 0:10                            1.566642   
2016/1/1 0:15                            1.707677   
2016/1/1 0:20                            1.746989   

               Radiation Global Tilted(W/m²)  Radiation Diffuse Tilted(W/m²)  \
Timestamp                                                                      
2016/1/1 0:00                       1.868883                        1.853108   
2016/1/1 0:05                       1.803461                        1.796472   
2016/1/1 0:10                       2.339108                        2.343814   
2016/1/1 0:15                       2.719022                        2.654432   
2016/1/1 0:20                       2.489688                        2.495872   

               ...  Station_2  Station_3  Station_4  Station_5  Station_6  \
Timestamp      ...                                                          
2016/1/1 0:00  ...        0.0   0.000133   0.000400        0.0        0.0   
2016/1/1 0:05  ...        0.0   0.000200   0.000400        0.0        0.0   
2016/1/1 0:10  ...        0.0   0.000200   0.000533        0.0        0.0   
2016/1/1 0:15  ...        0.0   0.000067   0.000067        0.0        0.0   
2016/1/1 0:20  ...        0.0   0.000267   0.000600        0.0        0.0   

               Station_7     Station_8  Station_9  DKA.Meter1_power  \
Timestamp                                                             
2016/1/1 0:00      -0.02  2.000000e-04        0.0               NaN   
2016/1/1 0:05      -0.02  2.000000e-04        0.0               NaN   
2016/1/1 0:10      -0.02  1.333330e-04        0.0               NaN   
2016/1/1 0:15      -0.02  0.000000e+00        0.0               NaN   
2016/1/1 0:20      -0.02  1.460000e-11        0.0               NaN   

               DKA.Meter2_power  
Timestamp                        
2016/1/1 0:00         -0.233352  
2016/1/1 0:05         -0.243912  
2016/1/1 0:10         -0.241038  
2016/1/1 0:15         -0.218567  
2016/1/1 0:20         -0.217211  

[5 rows x 21 columns]

In [19]:
strsplit = dataset.Timestamp.str.split(expand = True)
strsplit.rename(columns={0:'date',1:'time'},inplace=True)
date = strsplit.date.str.split(expand = True,pat='/')
date.rename(columns={0:'Year',1:'Month',2:'Day'},inplace=True)
time = strsplit.time.str.split(expand = True,pat=':')
#time.rename(columns = {0:'H',1:'M',2:'S'},inplace = True)
time.rename(columns = {0:'Hour',1:'Minutes'},inplace = True)
dataset = pd.concat([dataset,date,time],axis=1)
dataset.columns

Index(['Timestamp', 'Temperature(°C)', 'Relative Humidity(%)',
       'Wind Speed(m/s)', 'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Radiation Global Tilted(W/m²)',
       'Radiation Diffuse Tilted(W/m²)', 'Station_1', 'Station_2', 'Station_3',
       'Station_4', 'Station_5', 'Station_6', 'Station_7', 'Station_8',
       'Station_9', 'DKA.Meter1_power', 'DKA.Meter2_power', 'Year', 'Month',
       'Day', 'Hour', 'Minutes'],
      dtype='object')

In [20]:
#训练集重新排布列
dataset=dataset[['Timestamp','Year', 'Month',
       'Day', 'Hour', 'Minutes', 'Temperature(°C)', 'Relative Humidity(%)',
       'Wind Speed(m/s)', 'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Radiation Global Tilted(W/m²)',
       'Radiation Diffuse Tilted(W/m²)', 'Station_1', 'Station_2', 'Station_3',
       'Station_4', 'Station_5', 'Station_6', 'Station_7', 'Station_8',
       'Station_9', 'DKA.Meter1_power', 'DKA.Meter2_power']]


print(dataset.head(5))

                   Timestamp  Year Month Day Hour Minutes  Temperature(°C)  \
Timestamp                                                                    
2016/1/1 0:00  2016/1/1 0:00  2016     1   1    0      00        17.176111   
2016/1/1 0:05  2016/1/1 0:05  2016     1   1    0      05        17.133446   
2016/1/1 0:10  2016/1/1 0:10  2016     1   1    0      10        17.084999   
2016/1/1 0:15  2016/1/1 0:15  2016     1   1    0      15        16.955778   
2016/1/1 0:20  2016/1/1 0:20  2016     1   1    0      20        16.854271   

               Relative Humidity(%)  Wind Speed(m/s)  Wind Direction(Degrees)  \
Timestamp                                                                       
2016/1/1 0:00             87.095665         2.421402               186.941284   
2016/1/1 0:05             87.671669         2.688986               191.876770   
2016/1/1 0:10             88.563240         2.672599               188.268082   
2016/1/1 0:15             90.001663         3.78

In [21]:
dataset.to_csv('../Dataset/raw_DKA.Station_1-9.csv',index=False)

# 丢弃夜间数据

In [22]:
# # #数据类型转换
# train[['Year', 'Month','Day','Hour','Minutes']] = train[['Year', 'Month','Day','Hour','Minutes']].astype(int)
# test[['Year', 'Month','Day','Hour','Minutes']] = test[['Year', 'Month','Day','Hour','Minutes']].astype(int)

# #丢弃
# train = train.drop(train[(train.Hour < 7) | (train.Hour > 18)].index)
# test = test.drop(test[(test.Hour < 7) | (test.Hour > 18)].index)

# #数据类型还原
# train[['Year', 'Month','Day','Hour','Minutes']] = train[['Year', 'Month','Day','Hour','Minutes']].astype(object)
# test[['Year', 'Month','Day','Hour','Minutes']] = test[['Year', 'Month','Day','Hour','Minutes']].astype(object)


# 检测、剔除存在无效的空行、空列

In [23]:
#筛选有空的所有行
NaNdata = dataset.loc[dataset["Station_1"].isnull(),:]
NaNdata

Timestamp  Year Month Day Hour Minutes  \
Timestamp                                                         
2016/1/7 1:05        2016/1/7 1:05  2016     1   7    1      05   
2016/1/7 1:10        2016/1/7 1:10  2016     1   7    1      10   
2016/1/7 1:15        2016/1/7 1:15  2016     1   7    1      15   
2016/1/7 1:20        2016/1/7 1:20  2016     1   7    1      20   
2016/1/7 1:25        2016/1/7 1:25  2016     1   7    1      25   
2016/1/7 1:30        2016/1/7 1:30  2016     1   7    1      30   
2016/1/7 1:35        2016/1/7 1:35  2016     1   7    1      35   
2016/1/7 1:40        2016/1/7 1:40  2016     1   7    1      40   
2016/1/7 1:45        2016/1/7 1:45  2016     1   7    1      45   
2016/1/7 1:50        2016/1/7 1:50  2016     1   7    1      50   
2016/1/7 1:55        2016/1/7 1:55  2016     1   7    1      55   
2016/1/7 2:00        2016/1/7 2:00  2016     1   7    2      00   
2016/1/7 2:05        2016/1/7 2:05  2016     1   7    2      05   
2016/1/7 2:10        2016/1/7 2:10  2016     1   7    2      10   
2016/3/5 0:00        2016/3/5 0:00  2016     3   5    0      00   
2016/3/12 0:00      2016/3/12 0:00  2016     3  12    0      00   
2016/3/19 0:00      2016/3/19 0:00  2016     3  19    0      00   
2016/3/26 0:00      2016/3/26 0:00  2016     3  26    0      00   
2016/4/2 0:00        2016/4/2 0:00  2016     4   2    0      00   
2016/4/9 0:00        2016/4/9 0:00  2016     4   9    0      00   
2016/4/11 16:30    2016/4/11 16:30  2016     4  11   16      30   
2016/4/11 22:35    2016/4/11 22:35  2016     4  11   22      35   
2016/4/12 9:30      2016/4/12 9:30  2016     4  12    9      30   
2016/4/15 11:00    2016/4/15 11:00  2016     4  15   11      00   
2016/4/15 11:05    2016/4/15 11:05  2016     4  15   11      05   
2016/4/15 11:10    2016/4/15 11:10  2016     4  15   11      10   
2016/4/15 11:15    2016/4/15 11:15  2016     4  15   11      15   
2016/4/15 11:20    2016/4/15 11:20  2016     4  15   11      20   
2016/4/15 11:25    2016/4/15 11:25  2016     4  15   11      25   
2016/4/15 11:30    2016/4/15 11:30  2016     4  15   11      30   
...                            ...   ...   ...  ..  ...     ...   
2019/11/25 4:30    2019/11/25 4:30  2019    11  25    4      30   
2019/11/25 5:15    2019/11/25 5:15  2019    11  25    5      15   
2019/11/25 5:30    2019/11/25 5:30  2019    11  25    5      30   
2019/11/25 5:45    2019/11/25 5:45  2019    11  25    5      45   
2019/11/25 6:15    2019/11/25 6:15  2019    11  25    6      15   
2019/11/25 6:30    2019/11/25 6:30  2019    11  25    6      30   
2019/11/25 7:00    2019/11/25 7:00  2019    11  25    7      00   
2019/11/25 7:15    2019/11/25 7:15  2019    11  25    7      15   
2019/11/25 7:45    2019/11/25 7:45  2019    11  25    7      45   
2019/11/25 8:00    2019/11/25 8:00  2019    11  25    8      00   
2019/11/25 8:30    2019/11/25 8:30  2019    11  25    8      30   
2019/11/25 8:45    2019/11/25 8:45  2019    11  25    8      45   
2019/11/25 9:15    2019/11/25 9:15  2019    11  25    9      15   
2019/11/25 9:30    2019/11/25 9:30  2019    11  25    9      30   
2019/11/25 10:15  2019/11/25 10:15  2019    11  25   10      15   
2019/11/28 0:05    2019/11/28 0:05  2019    11  28    0      05   
2019/11/28 0:10    2019/11/28 0:10  2019    11  28    0      10   
2019/11/28 0:15    2019/11/28 0:15  2019    11  28    0      15   
2019/11/28 0:20    2019/11/28 0:20  2019    11  28    0      20   
2019/11/28 0:25    2019/11/28 0:25  2019    11  28    0      25   
2019/11/30 0:00    2019/11/30 0:00  2019    11  30    0      00   
2019/12/1 2:30      2019/12/1 2:30  2019    12   1    2      30   
2019/12/3 7:15      2019/12/3 7:15  2019    12   3    7      15   
2019/12/3 8:15      2019/12/3 8:15  2019    12   3    8      15   
2019/12/6 2:30      2019/12/6 2:30  2019    12   6    2      30   
2019/12/7 0:00      2019/12/7 0:00  2019    12   7    0      00   
2019/12/14 0:00    2019/12/14 0:00  2019    12  14    0      00   
2019/

# 导出数据有空值的情况

In [24]:
NaNdata.to_csv('../Dataset/NaN_DKA.Station.csv',index=False)

# 训练集去掉全是空值的行和列

In [25]:
#训练集去掉全是空值的行和列    #axis=0, how='any', thresh=None, subset=None, inplace=False)
dataset.dropna(axis="index",thresh=16 , inplace=True)  #删除掉全是空值的行
dataset.dropna(axis="columns", how='all', inplace=True)  #删除掉全是空值的列

# #测试集去掉全是空值的行和列
# test.dropna(axis="index", thresh=22, inplace=True)  #删除掉全是空值的行
# test.dropna(axis="columns", how='all', inplace=True)  #删除掉全是空值的列

In [26]:
dataset.loc[dataset["Station_1"].isnull(),:]

Timestamp  Year Month Day Hour Minutes  Temperature(°C)  \
Timestamp                                                                    
2016/1/7 1:30  2016/1/7 1:30  2016     1   7    1      30        21.472599   
2016/1/7 1:35  2016/1/7 1:35  2016     1   7    1      35        21.517189   
2016/1/7 1:40  2016/1/7 1:40  2016     1   7    1      40        21.338928   
2016/1/7 1:45  2016/1/7 1:45  2016     1   7    1      45        21.207865   
2016/1/7 1:50  2016/1/7 1:50  2016     1   7    1      50        21.157518   
2016/1/7 1:55  2016/1/7 1:55  2016     1   7    1      55        21.202934   
2016/1/7 2:00  2016/1/7 2:00  2016     1   7    2      00        21.211445   
2016/1/7 2:05  2016/1/7 2:05  2016     1   7    2      05        21.108192   
2016/1/7 2:10  2016/1/7 2:10  2016     1   7    2      10        21.269835   

               Relative Humidity(%)  Wind Speed(m/s)  Wind Direction(Degrees)  \
Timestamp                                                                       
2016/1/7 1:30             36.849094         8.039104                68.051712   
2016/1/7 1:35             34.722862         6.995687                72.694733   
2016/1/7 1:40             38.950737         6.042944                69.767342   
2016/1/7 1:45             41.284744         7.528507                57.747742   
2016/1/7 1:50             39.164032         8.289650                64.676605   
2016/1/7 1:55             37.092865         6.819695                74.499115   
2016/1/7 2:00             40.735878         4.895438                56.317230   
2016/1/7 2:05             40.645844         4.835287                41.207649   
2016/1/7 2:10             40.922939         6.509986                44.867706   

               ...  Station_2  Station_3  Station_4  Station_5  Station_6  \
Timestamp      ...                                                          
2016/1/7 1:30  ...        0.0        NaN   0.000133        0.0        0.0   
2016/1/7 1:35  ...        0.0        NaN   0.000000        0.0        0.0   
2016/1/7 1:40  ...        0.0        NaN   0.000400        0.0        0.0   
2016/1/7 1:45  ...        0.0        NaN   0.000267        0.0        0.0   
2016/1/7 1:50  ...        0.0        NaN   0.000133        0.0        0.0   
2016/1/7 1:55  ...        0.0        NaN   0.000200        0.0        0.0   
2016/1/7 2:00  ...        0.0        NaN   0.000400        0.0        0.0   
2016/1/7 2:05  ...        0.0        NaN   0.000667        0.0        0.0   
2016/1/7 2:10  ...        0.0        NaN   0.000267        0.0        0.0   

               Station_7  Station_8  Station_9  DKA.Meter1_power  \
Timestamp                                                          
2016/1/7 1:30        NaN  -0.000067        NaN               NaN   
2016/1/7 1:35        NaN  -0.000200        NaN               NaN   
2016/1/7 1:40        NaN   0.000200        NaN               NaN   
2016/1/7 1:45        NaN  -0.000200        NaN               NaN   
2016/1/7 1:50        NaN   0.000200        NaN               NaN   
2016/1/7 1:55        NaN   0.000133        NaN               NaN   
2016/1/7 2:00        NaN  -0.000267        NaN               NaN   
2016/1/7 2:05        NaN  -0.000133        NaN               NaN   
2016/1/7 2:10        NaN   0.000200        NaN               NaN   

               DKA.Meter2_power  
Timestamp                        
2016/1/7 1:30               NaN  
2016/1/7 1:35               NaN  
2016/1/7 1:40               NaN  
2016/1/7 1:45               NaN  
2016/1/7 1:50               NaN  
2016/1/7 1:55               NaN  
2016/1/7 2:00               NaN  
2016/1/7 2:05               NaN  
2016/1/7 2:10               NaN  

[9 rows x 26 columns]

In [27]:
dataset.columns

Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Speed(m/s)',
       'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Radiation Global Tilted(W/m²)',
       'Radiation Diffuse Tilted(W/m²)', 'Station_1', 'Station_2', 'Station_3',
       'Station_4', 'Station_5', 'Station_6', 'Station_7', 'Station_8',
       'Station_9', 'DKA.Meter1_power', 'DKA.Meter2_power'],
      dtype='object')

In [28]:
#检测训练集是否去掉全空的行和列
detectioncolumns=['Temperature(°C)', 'Relative Humidity(%)', 'Wind Speed(m/s)',
       'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Radiation Global Tilted(W/m²)',
       'Radiation Diffuse Tilted(W/m²)', 'Station_1', 'Station_2', 'Station_3',
       'Station_4', 'Station_5', 'Station_6', 'Station_7', 'Station_8',
       'Station_9', 'DKA.Meter1_power', 'DKA.Meter2_power']
for i in detectioncolumns:
    print("训练集",i,"列去掉全空")
    print(dataset.loc[dataset[i].isnull(),:])
# #检测训练集是否去掉全空的行和列
# for i in detectioncolumns:
#     print("测试集",i,"列去掉全空")
#     print(test.loc[test[i].isnull(),:])

训练集 Temperature(°C) 列去掉全空
Empty DataFrame
Columns: [Timestamp, Year, Month, Day, Hour, Minutes, Temperature(°C), Relative Humidity(%), Wind Speed(m/s), Wind Direction(Degrees), Daily Rainfall(mm), Global Horizontal Radiation(W/m²), Diffuse Horizontal Radiation(W/m²), Radiation Global Tilted(W/m²), Radiation Diffuse Tilted(W/m²), Station_1, Station_2, Station_3, Station_4, Station_5, Station_6, Station_7, Station_8, Station_9, DKA.Meter1_power, DKA.Meter2_power]
Index: []

[0 rows x 26 columns]
训练集 Relative Humidity(%) 列去掉全空
Empty DataFrame
Columns: [Timestamp, Year, Month, Day, Hour, Minutes, Temperature(°C), Relative Humidity(%), Wind Speed(m/s), Wind Direction(Degrees), Daily Rainfall(mm), Global Horizontal Radiation(W/m²), Diffuse Horizontal Radiation(W/m²), Radiation Global Tilted(W/m²), Radiation Diffuse Tilted(W/m²), Station_1, Station_2, Station_3, Station_4, Station_5, Station_6, Station_7, Station_8, Station_9, DKA.Meter1_power, DKA.Meter2_power]
Index: []

[0 rows x 26 column

训练集 Wind Direction(Degrees) 列去掉全空
Empty DataFrame
Columns: [Timestamp, Year, Month, Day, Hour, Minutes, Temperature(°C), Relative Humidity(%), Wind Speed(m/s), Wind Direction(Degrees), Daily Rainfall(mm), Global Horizontal Radiation(W/m²), Diffuse Horizontal Radiation(W/m²), Radiation Global Tilted(W/m²), Radiation Diffuse Tilted(W/m²), Station_1, Station_2, Station_3, Station_4, Station_5, Station_6, Station_7, Station_8, Station_9, DKA.Meter1_power, DKA.Meter2_power]
Index: []

[0 rows x 26 columns]
训练集 Daily Rainfall(mm) 列去掉全空
Empty DataFrame
Columns: [Timestamp, Year, Month, Day, Hour, Minutes, Temperature(°C), Relative Humidity(%), Wind Speed(m/s), Wind Direction(Degrees), Daily Rainfall(mm), Global Horizontal Radiation(W/m²), Diffuse Horizontal Radiation(W/m²), Radiation Global Tilted(W/m²), Radiation Diffuse Tilted(W/m²), Station_1, Station_2, Station_3, Station_4, Station_5, Station_6, Station_7, Station_8, Station_9, DKA.Meter1_power, DKA.Meter2_power]
Index: []

[0 rows x 26 

                   Timestamp  Year Month Day Hour Minutes  Temperature(°C)  \
Timestamp                                                                    
2016/1/7 1:30  2016/1/7 1:30  2016     1   7    1      30        21.472599   
2016/1/7 1:35  2016/1/7 1:35  2016     1   7    1      35        21.517189   
2016/1/7 1:40  2016/1/7 1:40  2016     1   7    1      40        21.338928   
2016/1/7 1:45  2016/1/7 1:45  2016     1   7    1      45        21.207865   
2016/1/7 1:50  2016/1/7 1:50  2016     1   7    1      50        21.157518   
2016/1/7 1:55  2016/1/7 1:55  2016     1   7    1      55        21.202934   
2016/1/7 2:00  2016/1/7 2:00  2016     1   7    2      00        21.211445   
2016/1/7 2:05  2016/1/7 2:05  2016     1   7    2      05        21.108192   
2016/1/7 2:10  2016/1/7 2:10  2016     1   7    2      10        21.269835   

               Relative Humidity(%)  Wind Speed(m/s)  Wind Direction(Degrees)  \
Timestamp                                                   

Wind Speed(m/s) * 207972 x 26
Radiation Global Tilted(W/m²) * 57122 x 26
Radiation Diffuse Tilted(W/m²) 57122 rows x 26 columns
DKA.Meter1_power * 26194 x 26
DKA.Meter2_power * 14517 x 26

In [29]:
dataset.columns

Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Speed(m/s)',
       'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Radiation Global Tilted(W/m²)',
       'Radiation Diffuse Tilted(W/m²)', 'Station_1', 'Station_2', 'Station_3',
       'Station_4', 'Station_5', 'Station_6', 'Station_7', 'Station_8',
       'Station_9', 'DKA.Meter1_power', 'DKA.Meter2_power'],
      dtype='object')

In [30]:
dataset = dataset.drop(['Wind Speed(m/s)','Radiation Global Tilted(W/m²)',
       'Radiation Diffuse Tilted(W/m²)'],axis=1)

In [31]:
dataset.columns

Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Direction(Degrees)',
       'Daily Rainfall(mm)', 'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Station_1', 'Station_2',
       'Station_3', 'Station_4', 'Station_5', 'Station_6', 'Station_7',
       'Station_8', 'Station_9', 'DKA.Meter1_power', 'DKA.Meter2_power'],
      dtype='object')

In [32]:
dataset['Station_3'] = dataset['Station_3'].fillna(method='ffill')

In [33]:
dataset.loc[dataset["Temperature(°C)"].isnull(),:]

Empty DataFrame
Columns: [Timestamp, Year, Month, Day, Hour, Minutes, Temperature(°C), Relative Humidity(%), Wind Direction(Degrees), Daily Rainfall(mm), Global Horizontal Radiation(W/m²), Diffuse Horizontal Radiation(W/m²), Station_1, Station_2, Station_3, Station_4, Station_5, Station_6, Station_7, Station_8, Station_9, DKA.Meter1_power, DKA.Meter2_power]
Index: []

[0 rows x 23 columns]

In [34]:
fillcolumns=['Temperature(°C)', 'Relative Humidity(%)',
       'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Station_1', 'Station_2', 'Station_3',
       'Station_4', 'Station_5', 'Station_6', 'Station_7', 'Station_8',
       'Station_9']
for i in fillcolumns:
    dataset[i] = dataset[i].fillna(method='ffill')
#     print("训练集",i,"列去掉全空")
    print(dataset.loc[dataset[i].isnull(),:])

Empty DataFrame
Columns: [Timestamp, Year, Month, Day, Hour, Minutes, Temperature(°C), Relative Humidity(%), Wind Direction(Degrees), Daily Rainfall(mm), Global Horizontal Radiation(W/m²), Diffuse Horizontal Radiation(W/m²), Station_1, Station_2, Station_3, Station_4, Station_5, Station_6, Station_7, Station_8, Station_9, DKA.Meter1_power, DKA.Meter2_power]
Index: []

[0 rows x 23 columns]
Empty DataFrame
Columns: [Timestamp, Year, Month, Day, Hour, Minutes, Temperature(°C), Relative Humidity(%), Wind Direction(Degrees), Daily Rainfall(mm), Global Horizontal Radiation(W/m²), Diffuse Horizontal Radiation(W/m²), Station_1, Station_2, Station_3, Station_4, Station_5, Station_6, Station_7, Station_8, Station_9, DKA.Meter1_power, DKA.Meter2_power]
Index: []

[0 rows x 23 columns]
Empty DataFrame
Columns: [Timestamp, Year, Month, Day, Hour, Minutes, Temperature(°C), Relative Humidity(%), Wind Direction(Degrees), Daily Rainfall(mm), Global Horizontal Radiation(W/m²), Diffuse Horizontal Radiat

In [35]:
#训练集和测试的维度
print("训练集维度：",dataset.shape)
# print("测试集维度：",test.shape)

训练集维度： (417731, 23)


# 筛选数据去空值、填缺值、

查看统计数筛选非异常值 3σ原则又称为拉依达准则


In [36]:
#查看所有列的统计结果
print("训练集统计信息\n",dataset.describe())
#查看所有列的统计结果
# print("测试集统计信息\n",test.describe())

训练集统计信息
        Temperature(°C)  Relative Humidity(%)  Wind Direction(Degrees)  \
count    417731.000000         417731.000000            417731.000000   
mean         21.624916             34.237967               146.224190   
std          10.180806             22.647584               191.220148   
min         -39.987625              0.000000             -9836.644531   
25%          14.952995             16.733972                35.308371   
50%          22.225859             28.292578                92.879303   
75%          29.054525             47.174940               265.569687   
max          51.261726            102.896820             30152.339840   

       Daily Rainfall(mm)  Global Horizontal Radiation(W/m²)  \
count       417731.000000                      417731.000000   
mean             0.276158                         267.782600   
std              2.286298                         367.372435   
min              0.000000                           0.000000   
25%          

# 去异常值

规则

0 < GHR/DHR < 10

0 <= 风向 <= 360

0 <= 湿度 <= 100

-5.1 <= 温度 <= 45.8

降水量 > 0

In [37]:
dataset.columns

Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Direction(Degrees)',
       'Daily Rainfall(mm)', 'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Station_1', 'Station_2',
       'Station_3', 'Station_4', 'Station_5', 'Station_6', 'Station_7',
       'Station_8', 'Station_9', 'DKA.Meter1_power', 'DKA.Meter2_power'],
      dtype='object')

去风向异常值

0 <= 风向 <= 360

In [38]:
index = np.arange(dataset.shape[0])[(dataset['Wind Direction(Degrees)'] < 0 ) | ( dataset['Wind Direction(Degrees)'] > 360)]
print(dataset.iloc[index])
dataset['Wind Direction(Degrees)'].iloc[index]=np.nan
print(dataset.loc[dataset['Wind Direction(Degrees)'].isnull(),:])

                         Timestamp  Year Month Day Hour Minutes  \
Timestamp                                                         
2016/1/2 23:50      2016/1/2 23:50  2016     1   2   23      50   
2016/1/3 8:00        2016/1/3 8:00  2016     1   3    8      00   
2016/1/3 9:40        2016/1/3 9:40  2016     1   3    9      40   
2016/1/4 19:30      2016/1/4 19:30  2016     1   4   19      30   
2016/1/5 20:30      2016/1/5 20:30  2016     1   5   20      30   
2016/1/6 6:40        2016/1/6 6:40  2016     1   6    6      40   
2016/1/8 1:10        2016/1/8 1:10  2016     1   8    1      10   
2016/1/8 1:15        2016/1/8 1:15  2016     1   8    1      15   
2016/1/8 3:05        2016/1/8 3:05  2016     1   8    3      05   
2016/1/10 13:45    2016/1/10 13:45  2016     1  10   13      45   
2016/1/11 2:00      2016/1/11 2:00  2016     1  11    2      00   
2016/1/11 7:10      2016/1/11 7:10  2016     1  11    7      10   
2016/1/11 22:50    2016/1/11 22:50  2016     1  11   22      5

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


去湿度异常值

0 <= 湿度 <= 100

In [39]:
index = np.arange(dataset.shape[0])[(dataset['Relative Humidity(%)'] < 0 ) | ( dataset['Relative Humidity(%)'] > 100)]
print(dataset.iloc[index])
dataset['Relative Humidity(%)'].iloc[index]=np.nan
print(dataset.loc[dataset['Relative Humidity(%)'].isnull(),:])

                     Timestamp  Year Month Day Hour Minutes  Temperature(°C)  \
Timestamp                                                                      
2016/1/1 6:15    2016/1/1 6:15  2016     1   1    6      15        16.502680   
2016/1/1 6:20    2016/1/1 6:20  2016     1   1    6      20        16.485319   
2016/1/1 6:25    2016/1/1 6:25  2016     1   1    6      25        16.491226   
2016/1/1 6:30    2016/1/1 6:30  2016     1   1    6      30        16.494886   
2016/1/1 6:35    2016/1/1 6:35  2016     1   1    6      35        16.518930   
2016/1/1 6:40    2016/1/1 6:40  2016     1   1    6      40        16.532946   
2016/1/1 6:45    2016/1/1 6:45  2016     1   1    6      45        16.499128   
2016/1/1 6:50    2016/1/1 6:50  2016     1   1    6      50        16.541285   
2016/1/1 6:55    2016/1/1 6:55  2016     1   1    6      55        16.555387   
2016/1/1 7:00    2016/1/1 7:00  2016     1   1    7      00        16.618614   
2016/1/1 7:05    2016/1/1 7:05  2016    

                     Timestamp  Year Month Day Hour Minutes  Temperature(°C)  \
Timestamp                                                                      
2016/1/1 6:15    2016/1/1 6:15  2016     1   1    6      15        16.502680   
2016/1/1 6:20    2016/1/1 6:20  2016     1   1    6      20        16.485319   
2016/1/1 6:25    2016/1/1 6:25  2016     1   1    6      25        16.491226   
2016/1/1 6:30    2016/1/1 6:30  2016     1   1    6      30        16.494886   
2016/1/1 6:35    2016/1/1 6:35  2016     1   1    6      35        16.518930   
2016/1/1 6:40    2016/1/1 6:40  2016     1   1    6      40        16.532946   
2016/1/1 6:45    2016/1/1 6:45  2016     1   1    6      45        16.499128   
2016/1/1 6:50    2016/1/1 6:50  2016     1   1    6      50        16.541285   
2016/1/1 6:55    2016/1/1 6:55  2016     1   1    6      55        16.555387   
2016/1/1 7:00    2016/1/1 7:00  2016     1   1    7      00        16.618614   
2016/1/1 7:05    2016/1/1 7:05  2016    

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


去温度异常值
-5.1 <= 温度 <= 45.8

In [40]:
index = np.arange(dataset.shape[0])[(dataset['Temperature(°C)'] < -5.1 ) | ( dataset['Temperature(°C)'] > 45.8)]
print(dataset.iloc[index])
dataset['Temperature(°C)'].iloc[index]=np.nan
print(dataset.loc[dataset['Temperature(°C)'].isnull(),:])

                       Timestamp  Year Month Day Hour Minutes  \
Timestamp                                                       
2016/7/5 13:00    2016/7/5 13:00  2016     7   5   13      00   
2016/7/5 13:05    2016/7/5 13:05  2016     7   5   13      05   
2016/7/5 13:15    2016/7/5 13:15  2016     7   5   13      15   
2016/7/5 13:20    2016/7/5 13:20  2016     7   5   13      20   
2016/7/5 13:25    2016/7/5 13:25  2016     7   5   13      25   
2018/3/18 12:25  2018/3/18 12:25  2018     3  18   12      25   
2018/3/18 13:35  2018/3/18 13:35  2018     3  18   13      35   
2018/3/18 13:40  2018/3/18 13:40  2018     3  18   13      40   
2018/3/18 13:45  2018/3/18 13:45  2018     3  18   13      45   
2018/3/18 13:50  2018/3/18 13:50  2018     3  18   13      50   
2018/3/18 13:55  2018/3/18 13:55  2018     3  18   13      55   
2018/3/18 14:00  2018/3/18 14:00  2018     3  18   14      00   
2018/3/18 14:05  2018/3/18 14:05  2018     3  18   14      05   
2018/3/18 14:10  2018/3/1

                       Timestamp  Year Month Day Hour Minutes  \
Timestamp                                                       
2016/7/5 13:00    2016/7/5 13:00  2016     7   5   13      00   
2016/7/5 13:05    2016/7/5 13:05  2016     7   5   13      05   
2016/7/5 13:15    2016/7/5 13:15  2016     7   5   13      15   
2016/7/5 13:20    2016/7/5 13:20  2016     7   5   13      20   
2016/7/5 13:25    2016/7/5 13:25  2016     7   5   13      25   
2018/3/18 12:25  2018/3/18 12:25  2018     3  18   12      25   
2018/3/18 13:35  2018/3/18 13:35  2018     3  18   13      35   
2018/3/18 13:40  2018/3/18 13:40  2018     3  18   13      40   
2018/3/18 13:45  2018/3/18 13:45  2018     3  18   13      45   
2018/3/18 13:50  2018/3/18 13:50  2018     3  18   13      50   
2018/3/18 13:55  2018/3/18 13:55  2018     3  18   13      55   
2018/3/18 14:00  2018/3/18 14:00  2018     3  18   14      00   
2018/3/18 14:05  2018/3/18 14:05  2018     3  18   14      05   
2018/3/18 14:10  2018/3/1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


取辐照度异常值
0 < GHR/DHR < 10

In [41]:
index = np.arange(dataset.shape[0])[(dataset['Global Horizontal Radiation(W/m²)']/dataset['Diffuse Horizontal Radiation(W/m²)'] < 0 ) | ( dataset['Global Horizontal Radiation(W/m²)']/dataset['Diffuse Horizontal Radiation(W/m²)'] > 20)]
print(dataset.iloc[index])
dataset['Global Horizontal Radiation(W/m²)'].iloc[index]=np.nan
dataset['Diffuse Horizontal Radiation(W/m²)'].iloc[index]=np.nan
print(dataset.loc[dataset['Global Horizontal Radiation(W/m²)'].isnull(),:])

                         Timestamp  Year Month Day Hour Minutes  \
Timestamp                                                         
2016/9/22 7:40      2016/9/22 7:40  2016     9  22    7      40   
2017/10/24 17:05  2017/10/24 17:05  2017    10  24   17      05   

                  Temperature(°C)  Relative Humidity(%)  \
Timestamp                                                 
2016/9/22 7:40          11.069950             70.131241   
2017/10/24 17:05        36.896225              7.268924   

                  Wind Direction(Degrees)  Daily Rainfall(mm)  ...  Station_2  \
Timestamp                                                      ...              
2016/9/22 7:40                  71.507904                 0.0  ...  10.353034   
2017/10/24 17:05               345.645081                 0.0  ...  10.332733   

                  Station_3  Station_4  Station_5  Station_6  Station_7  \
Timestamp                                                                 
2016/9/22 7:40     

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [42]:
index = np.arange(dataset.shape[0])[ dataset['Daily Rainfall(mm)'] < 0  ]
print(dataset.iloc[index])

Empty DataFrame
Columns: [Timestamp, Year, Month, Day, Hour, Minutes, Temperature(°C), Relative Humidity(%), Wind Direction(Degrees), Daily Rainfall(mm), Global Horizontal Radiation(W/m²), Diffuse Horizontal Radiation(W/m²), Station_1, Station_2, Station_3, Station_4, Station_5, Station_6, Station_7, Station_8, Station_9, DKA.Meter1_power, DKA.Meter2_power]
Index: []

[0 rows x 23 columns]


In [43]:
# # 定义3σ法则识别异常值函数 设为nan 空值
# def three_sigma(Ser1):
#     '''
#     Ser1：表示传入DataFrame的某一列。
#     '''
#     rule = (Ser1.mean()- 3*Ser1.std()>Ser1) | (Ser1.mean()+ 3*Ser1.std()< Ser1)
#     index = np.arange(Ser1.shape[0])[rule]
#     outrange = Ser1.iloc[index]
#     return outrange,index
# detectioncolumns=['WTC', 'WRH', 'GHR', 'DHR', 'WD', 'WDR', 'RGT', 'RDT']
# for i in detectioncolumns:
#     (errorrange,errorindex)=three_sigma(train[i])
#     print(errorrange)
#     train[i].iloc[errorindex]=np.nan
#     print(train[i].iloc[errorindex])
#     print(train.iloc[errorindex])


In [44]:
#使用df.isnull().sum()来获得与行数None和NaN值。
dataset.isnull().sum()

Timestamp                                 0
Year                                      0
Month                                     0
Day                                       0
Hour                                      0
Minutes                                   0
Temperature(°C)                         644
Relative Humidity(%)                   3904
Wind Direction(Degrees)                1116
Daily Rainfall(mm)                        0
Global Horizontal Radiation(W/m²)         2
Diffuse Horizontal Radiation(W/m²)        2
Station_1                                 0
Station_2                                 0
Station_3                                 0
Station_4                                 0
Station_5                                 0
Station_6                                 0
Station_7                                 0
Station_8                                 0
Station_9                                 0
DKA.Meter1_power                      26203
DKA.Meter2_power                

# 向下填充缺失值

In [45]:
# fillcolumns=['Temperature(°C)', 'Relative Humidity(%)',
#        'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
#        'Global Horizontal Radiation(W/m²)',
#        'Diffuse Horizontal Radiation(W/m²)', 'Station_1', 'Station_2', 'Station_3',
#        'Station_4', 'Station_5', 'Station_6', 'Station_7', 'Station_8',
#        'Station_9']
fillcolumns=['Temperature(°C)', 'Relative Humidity(%)',
       'Wind Direction(Degrees)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)']
for i in fillcolumns:
    dataset[i] = dataset[i].fillna(method='ffill')
#     print("训练集",i,"列去掉全空")
    print(dataset.loc[dataset[i].isnull(),:])

Empty DataFrame
Columns: [Timestamp, Year, Month, Day, Hour, Minutes, Temperature(°C), Relative Humidity(%), Wind Direction(Degrees), Daily Rainfall(mm), Global Horizontal Radiation(W/m²), Diffuse Horizontal Radiation(W/m²), Station_1, Station_2, Station_3, Station_4, Station_5, Station_6, Station_7, Station_8, Station_9, DKA.Meter1_power, DKA.Meter2_power]
Index: []

[0 rows x 23 columns]
Empty DataFrame
Columns: [Timestamp, Year, Month, Day, Hour, Minutes, Temperature(°C), Relative Humidity(%), Wind Direction(Degrees), Daily Rainfall(mm), Global Horizontal Radiation(W/m²), Diffuse Horizontal Radiation(W/m²), Station_1, Station_2, Station_3, Station_4, Station_5, Station_6, Station_7, Station_8, Station_9, DKA.Meter1_power, DKA.Meter2_power]
Index: []

[0 rows x 23 columns]
Empty DataFrame
Columns: [Timestamp, Year, Month, Day, Hour, Minutes, Temperature(°C), Relative Humidity(%), Wind Direction(Degrees), Daily Rainfall(mm), Global Horizontal Radiation(W/m²), Diffuse Horizontal Radiat

In [46]:
dataset.isnull().sum()

Timestamp                                 0
Year                                      0
Month                                     0
Day                                       0
Hour                                      0
Minutes                                   0
Temperature(°C)                           0
Relative Humidity(%)                      0
Wind Direction(Degrees)                   0
Daily Rainfall(mm)                        0
Global Horizontal Radiation(W/m²)         0
Diffuse Horizontal Radiation(W/m²)        0
Station_1                                 0
Station_2                                 0
Station_3                                 0
Station_4                                 0
Station_5                                 0
Station_6                                 0
Station_7                                 0
Station_8                                 0
Station_9                                 0
DKA.Meter1_power                      26203
DKA.Meter2_power                

# 电站功率KW 变为W

In [47]:
# kw_to_wcolumns=['Station_1', 'Station_2', 'Station_3','Station_4', 'Station_5', 'Station_6', 'Station_7', 'Station_8','Station_9']
# for i in kw_to_wcolumns:
#     dataset[i] = dataset[i]*1000 

# 处理完成数据保存

In [48]:
dataset.to_csv('../Dataset/DKA.Station_1-9.csv',index=False)

# 训练集和测试集批量化数据保存

In [49]:
dataset.columns

Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Direction(Degrees)',
       'Daily Rainfall(mm)', 'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Station_1', 'Station_2',
       'Station_3', 'Station_4', 'Station_5', 'Station_6', 'Station_7',
       'Station_8', 'Station_9', 'DKA.Meter1_power', 'DKA.Meter2_power'],
      dtype='object')

In [50]:
dataset.dtypes

Timestamp                              object
Year                                   object
Month                                  object
Day                                    object
Hour                                   object
Minutes                                object
Temperature(°C)                       float64
Relative Humidity(%)                  float64
Wind Direction(Degrees)               float64
Daily Rainfall(mm)                    float64
Global Horizontal Radiation(W/m²)     float64
Diffuse Horizontal Radiation(W/m²)    float64
Station_1                             float64
Station_2                             float64
Station_3                             float64
Station_4                             float64
Station_5                             float64
Station_6                             float64
Station_7                             float64
Station_8                             float64
Station_9                             float64
DKA.Meter1_power                  

In [51]:
dataset[['Year', 'Month', 'Day', 'Hour','Minutes']] = dataset[['Year', 'Month', 'Day', 'Hour','Minutes']].astype("int")

In [52]:
dataset.dtypes

Timestamp                              object
Year                                    int32
Month                                   int32
Day                                     int32
Hour                                    int32
Minutes                                 int32
Temperature(°C)                       float64
Relative Humidity(%)                  float64
Wind Direction(Degrees)               float64
Daily Rainfall(mm)                    float64
Global Horizontal Radiation(W/m²)     float64
Diffuse Horizontal Radiation(W/m²)    float64
Station_1                             float64
Station_2                             float64
Station_3                             float64
Station_4                             float64
Station_5                             float64
Station_6                             float64
Station_7                             float64
Station_8                             float64
Station_9                             float64
DKA.Meter1_power                  

In [53]:
train = dataset.loc[(dataset['Year'] == 2016 ) | (dataset['Year'] == 2017) | (dataset['Year'] == 2018),:]
test =  dataset.loc[(dataset['Year'] == 2019 ),:]

In [54]:
site = list(['Station_1', 'Station_2',
       'Station_3', 'Station_4', 'Station_5', 'Station_6', 'Station_7',
       'Station_8', 'Station_9'])

trainnames=list()
testnames=list()
trainfiles=list()
testfiles=list()

trainnames += [('train_%s' % (j) )for j in site]
testnames += [('test_%s' % (j) )for j in site]

trainfiles += [('../Dataset/train0702/%s.csv' % (j) ) for j in trainnames]
testfiles += [('../Dataset/test0702/%s.csv' % (j) ) for j in testnames]

for i in range(0, len(site)):
    trainnames[i]=train[['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Direction(Degrees)',
       'Daily Rainfall(mm)', 'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)',
       site[i]]]
    testnames[i]=test[['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Direction(Degrees)',
       'Daily Rainfall(mm)', 'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)',
       site[i]]]
    
    trainnames[i].to_csv(trainfiles[i],index=False)
    testnames[i].to_csv(testfiles[i],index=False)

# 截止到这里

去掉任何有空值的行

In [ ]:
#训练集去掉全是空值的行和列    #axis=0, how='any', thresh=None, subset=None, inplace=False)
# detectioncolumns=['WTC', 'WRH', 'GHR', 'DHR', 'WD', 'WDR', 'RGT', 'RDT']
# for i in detectioncolumns:
train.dropna(axis="index",how='any' , inplace=True)  #删除掉全是空值的行
# train.dropna(axis="columns", how='all', inplace=True)  #删除掉全是空值的列

test.dropna(axis="index", how='any', inplace=True)  #删除掉全是空值的行
# test.dropna(axis="columns", how='all', inplace=True)  #删除掉全是空值的列

In [ ]:
#使用df.isnull().sum()来获得与行数None和NaN值。
train.isnull().sum()

In [ ]:
test.isnull().sum()

# 填补缺失值

In [ ]:
#去训练集        某列缺失的值填充为 平均值
#train.ffill({"RGT":train['RGT'].mean()})
detectioncolumns=['WTC', 'WRH', 'GHR', 'DHR', 'WD', 'WDR', 'RGT', 'RDT']
for i in detectioncolumns:
    replace_NA=train[i].mean()
    train[i].fillna(replace_NA,inplace=True)
#去除测试集空值
for i in detectioncolumns:
    replace_NA=test[i].mean()
    test[i].fillna(replace_NA,inplace=True)


In [ ]:
#使用df.isnull().sum()来获得与行数None和NaN值。
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.columns

In [ ]:
#选取站点一
# train_1A=train.drop(columns=['Timestamp','1B Trina_power','2 eco_power','3 BP_power',
#         '4 Kyocera_power',
#         '5 Kyocera_power',
#          '6 Kyocera_power',
#          '7 First_power',
#          '8 Kaneka_power',
#         'DKA.Meter1_power',
#         'DKA.Meter2_power'],inplace=True)
train_1A=train[['Year', 'Month', 'Day', 'Hour', 'Minutes','WTC', 'WRH', 'GHR', 'DHR', 'WD', 'WDR', 'RGT', 'RDT',
       '1A Trina_power']]
train_1A.to_csv('../Dataset/train_1A.csv',index=True)
train_1A.head()

# test_1A=test.drop(columns=['Timestamp','1B Trina_power','2 eco_power','3 BP_power',
#         '4 Kyocera_power',
#         '5 Kyocera_power',
#          '6 Kyocera_power',
#          '7 First_power',
#          '8 Kaneka_power',
#         'DKA.Meter1_power',
#         'DKA.Meter2_power'],inplace=True)
test_1A=test[['Year', 'Month', 'Day', 'Hour', 'Minutes','WTC', 'WRH', 'GHR', 'DHR', 'WD', 'WDR', 'RGT', 'RDT',
       '1A Trina_power']]
test_1A.to_csv('../Dataset/test_1A.csv',index=True)
test_1A.head()

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
train.head()

In [ ]:
train_1A.head()

# 相关系数和协方差的计算

In [ ]:
train_1A.dtypes

In [ ]:
# #数据类型转换
# train_1A[['Year', 'Month','Day','Hour','Minutes']] = train_1A[['Year', 'Month','Day','Hour','Minutes']].astype(int)
# train_1A.dtypes

In [ ]:
train_1A.head()

In [ ]:
#协方差矩阵：衡量同向反向程度;
#如果协方差为正，说明X，Y同向变化，协方差越大说明同向程度越高;如果协方差为负，说明X，Y反向运动，协方差越小说明反向程度越高。
train_1A.cov()

In [ ]:
# 相关系数矩阵:衡量相似度程度;
#当他们的相关系数为1时，说明两个变量变化时的正向相似度最大，当相关系数为－1时，说明两个变量变化的反向相似度最大.
train_1A.corr()

'DKA.WeatherStation - Weather Temperature Celsius (°C)':'WTC',
       'DKA.WeatherStation - Weather Relative Humidity (%)':'WRH',
       'DKA.WeatherStation - Global Horizontal Radiation (W/m²)':'GHR',
       'DKA.WeatherStation - Diffuse Horizontal Radiation (W/m²)':'DHR',
       'DKA.WeatherStation - Wind Direction (Degrees)':'WD',
       'DKA.WeatherStation - Weather Daily Rainfall (mm)':'WDR',
       'DKA.WeatherStation - Radiation Global Tilted (W/m²)':'RGT',
       'DKA.WeatherStation - Radiation Diffuse Tilted (W/m²)':'RDT',

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# plt.figure(figsize=(6.5, 4)) 
x = train_1A['WTC']
y = train_1A['GHR']
z = train_1A['1A Trina_power']
ax = plt.subplot(projection='3d')  # 创建一个三维的绘图工程
ax.scatter(x, y, z,s=0.1,c='r')  # 绘制数据点,颜色是红色     ['c', 'b', 'g', 'r', 'm', 'y', 'k', 'w']
 
ax.set_zlabel('Power(kw)')  # 坐标轴
ax.set_xlabel('Weather Temperature Celsius (°C)')
ax.set_ylabel('Global Horizontal Radiation (W/m²)')
plt.yticks([0,400,800,1200,1600])
# plt.savefig('../picture/WTC_GHR_Power.svg',dpi=300,format='svg')
plt.savefig('../picture/WTC_GHR_Power.jpg')
plt.show()
# plt.savefig('../picture/WTC_GHR_Power.jpg')
# plt.draw()
# plt.pause(10)
# plt.savefig('../picture/3D.jpg')
# plt.close()
##############
# plt.figure(figsize=(6.5, 4)) 
x = train_1A['DHR']
y = train_1A['GHR']
z = train_1A['1A Trina_power']
ax = plt.subplot(projection='3d')  # 创建一个三维的绘图工程
ax.scatter(x, y, z,s=0.1,c='y')  # 绘制数据点,
 
ax.set_zlabel('Power(kw)')  # 坐标轴
ax.set_xlabel('Diffuse Horizontal Radiation (W/m²)')
ax.set_ylabel('Global Horizontal Radiation (W/m²)')
plt.yticks([0,400,800,1200,1600])
plt.savefig('../picture/DHR_GHR_Power.jpg')
plt.show()

######################
# plt.figure(figsize=(6.5, 4)) 
x = train_1A['WRH']
y = train_1A['GHR']
z = train_1A['1A Trina_power']
ax = plt.subplot(projection='3d')  # 创建一个三维的绘图工程
ax.scatter(x, y, z,s=0.1,c='m')  # 绘制数据点,颜
 
ax.set_zlabel('Power(kw)')  # 坐标轴
ax.set_xlabel('Weather Relative Humidity (%)')
ax.set_ylabel('Global Horizontal Radiation (W/m²)')
plt.yticks([0,400,800,1200,1600])
plt.savefig('../picture/WHR_GHR_Power.jpg')
plt.show()

########
# plt.figure(figsize=(6.5, 4)) 
x = train_1A['RGT']
y = train_1A['RDT']
z = train_1A['1A Trina_power']
ax = plt.subplot(projection='3d')  # 创建一个三维的绘图工程
ax.scatter(x, y, z,s=0.1,c='c')  # 绘制数据点,颜
 
ax.set_zlabel('Power(kw)')  # 坐标轴
ax.set_xlabel('Radiation Global Tilted (W/m²)')
ax.set_ylabel('Radiation Diffuse Tilted (W/m²)')
plt.yticks([0,400,800,1200,1600])
plt.savefig('../picture/RGT_RDT_Power.jpg')
plt.show()

